In [9]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier as KNN

In [10]:
data = pd.read_csv('./dataset/new/data.csv').drop(columns="Unnamed: 0")
labels = pd.read_csv('./dataset/new/labels.csv')

In [11]:
data.head()

,zcr_mean,zcr_std,yin_0,yin_1,yin_2,yin_3,yin_4,yin_5,yin_6,yin_7,...,cln_contrast_mean_4,cln_contrast_mean_5,cln_contrast_mean_6,cln_contrast_std_0,cln_contrast_std_1,cln_contrast_std_2,cln_contrast_std_3,cln_contrast_std_4,cln_contrast_std_5,cln_contrast_std_6
0,0.104213,0.025389,144.178630,167.02170,115.430466,115.579770,124.897190,124.318660,134.522060,138.791560,...,12.657701,14.509855,18.643116,2.905979,2.252765,2.997747,3.173423,2.699984,2.714874,2.329038
1,0.108329,0.018797,116.932335,116.81062,104.958510,106.856224,107.122960,131.222120,353.585820,128.099060,...,13.057363,13.359251,16.425354,2.510432,2.361842,2.691745,1.427239,2.369561,1.818932,2.163963
2,0.203055,0.022426,319.182280,131.99893,131.480270,121.196110,121.224365,121.071150,131.966660,794.266660,...,12.970959,21.755045,21.717041,1.537123,1.638238,3.026752,1.810459,2.805019,4.571950,4.293758
3,0.119210,0.032156,256.728060,515.78436,224.581180,112.526240,131.550890,131.923520,129.934570,115.061554,...,14.145430,15.075860,19.178572,2.318593,2.508579,1.901502,3.409710,2.446294,1.661938,3.035185
4,0.113839,0.019690,116.983660,108.42759,108.570360,136.068340,135.782400,111.091484,111.612175,104.272835,...,12.054803,14.731396,17.400953,3.947008,1.956441,2.477308,3.416498,1.927335,1.812949,1.570067


In [12]:
for i in range(7):
    print(f"bird{i}:",labels[labels["Aggregated"]==i].shape[0])

bird0: 85358
bird1: 5833
bird2: 8954
bird3: 8198
bird4: 2324
bird5: 5111
bird6: 4222


In [13]:
new_data = []
new_labels = []
for i in range(0,7):
    indices = labels[labels["Aggregated"] == i].sample(9000, replace=True).index.values.tolist()
    new_data.append(data.iloc[indices])
    new_labels.append(labels["Aggregated"].iloc[indices])
new_data = pd.concat(new_data)
new_labels = pd.concat(new_labels)

In [14]:
new_data.shape

(63000, 548)

In [15]:
for i in range(7):
    print(f"bird{i}:",new_labels[new_labels==i].shape[0])

bird0: 9000
bird1: 9000
bird2: 9000
bird3: 9000
bird4: 9000
bird5: 9000
bird6: 9000


In [ ]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB

kf = KFold(n_splits=5, shuffle=True, random_state=42)
gnb_acc = []
for train_index, test_index in kf.split(new_data):
    X_train, X_test = new_data.iloc[train_index], new_data.iloc[test_index]
    y_train, y_test = new_labels.iloc[train_index], new_labels.iloc[test_index]
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_predictions = gnb.predict(X_test)
    y_test[y_test == 0] = -1
    temp_acc = round(sum(np.where(y_test == y_predictions, 1, 0)) / len(y_test[y_test != -1]), 3)
    print(temp_acc)
    gnb_acc.append(temp_acc)
print(gnb_acc)
print(f"Estimate of the accuracy = {sum(gnb_acc) / len(gnb_acc)}")

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
acc = []
for train_index, test_index in kf.split(new_data):
    X_train, X_test = new_data.iloc[train_index], new_data.iloc[test_index]
    y_train, y_test = new_labels.iloc[train_index], new_labels.iloc[test_index]
    knn_cf = KNN(n_neighbors=6)
    knn_cf.fit(X_train, y_train)
    y_predictions = knn_cf.predict(X_test)
    y_test[y_test == 0] = -1
    temp_acc = sum(np.where(y_test == y_predictions, 1, 0)) / len(y_test[y_test != -1])
    print(temp_acc)
    acc.append(temp_acc)
print(acc)
print(f"Estimate of the accuracy = {sum(acc) / len(acc)}")

In [16]:
fss = new_data.copy()
corr = fss.corr()
plt.figure(figsize=(14,14))
sns.heatmap(corr)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
X_subset_features = new_data.copy()
correlation_matrix = X_subset_features.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
reduced_matrix = correlation_matrix.mask(mask)
feature_redundancy = reduced_matrix.abs().sum()
sorted_features = feature_redundancy.sort_values(ascending=False)

threshold = 0.9
selected_features = []
for feature in sorted_features.index:
    if not any(np.abs(correlation_matrix[feature][selected_features]) > threshold):
        selected_features.append(feature)

X_subset_features = X_subset_features[selected_features]